# Detection de motifs


## Boyer-Moore

In [1]:
import random
import numpy as np
import operator
import sys

In [2]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')
dico_nucl={"A":0,"T":1,"G":2,"C":3,0:"A",1:"T",2:"G",3:"C"}

pattern_len=8 #taille de motif
nb_subs= 1 #nb substituions
nb_sequences=20 #nb de sequences
len_sequences=100 #longuer des sequence

In [3]:
def generateRandomSequence():
    return "".join(random.choices(["A","T","G","C"],weights =[1,1,1,1],k=len_sequences))

def generateRandomSequences():
    return [generateRandomSequence() for k in range(0,nb_sequences)]

def generateRandomPattern():
    return "".join( random.choices(["A","T","G","C"],weights =[1,1,1,1],k=pattern_len) )

def generateVariation(pattern):
    dico_rand={0:"A",1:"T",2:"G",3:"C"}
    pattern_tmp=list(pattern)
    for i in range(0,nb_subs):
        pattern_tmp[int(random.uniform(0,len(pattern_tmp)))]=dico_rand[int(random.uniform(0,4))]
    return "".join(pattern_tmp)

def implantPatternSeq(seq,pattern):
    place_injection_pattern=int(random.uniform(0,len(seq)))
    return seq[:place_injection_pattern]+pattern+seq[place_injection_pattern:]

def implantMotif(sequences,pattern):
    return list(map(lambda seq:implantPatternSeq(seq,pattern),sequences))

def implantMotifSubstituions(sequences,pattern):
    return list(map(lambda seq:implantPatternSeq(seq,generateVariation(pattern)),sequences))
def motif_low_complexe_valide(motif,minrep,taille_motif):
    for mod in range(0,taille_motif):
        tmp_motif=motif[mod:]
        lettre_actu=tmp_motif[:taille_motif]
        cpt=1
        for i in range(taille_motif,len(tmp_motif),taille_motif):
                if lettre_actu==tmp_motif[i:i+taille_motif]:
                    cpt+=1
                
                    if (cpt>=minrep):
                        return False
                else:
                    lettre_actu=tmp_motif[i:i+taille_motif]
                    cpt=1
    return True
                

assert(motif_low_complexe_valide("ATGC",2,2)==True)
assert(motif_low_complexe_valide("GATATATGC",3,2)==False)
assert(motif_low_complexe_valide("ATATATGC",3,2)==False)
assert(motif_low_complexe_valide("ATAAGC",2,1)==False)
assert(motif_low_complexe_valide("ATGC",2,1)==True)
assert(motif_low_complexe_valide("ATTGC",2,1)==False)
assert(motif_low_complexe_valide("ATGC",50,1)==True)

                
def removeLowComplexe(sequences, minrep,taille_motif):
    res=[]
    
    for seq in sequences:
        res_seq=""
        for i in range(0,len(seq)-k,k):
            if motif_low_complexe_valide(seq[i:i+k],minrep,taille_motif):
                res_seq+=seq[i:i+k]
        
        if not res_seq=="":
            res+=[res_seq]
        
    
    return res

def implantMotifVar(k, v, t, n):
       
    sequences=generateRandomSequences(n, t)
    
    
    res=[]
    motif=generateMotif(k,v)
    
    for seq in sequences:
        res+=["".join(implant_motif_seq(seq,motif))]
    
    return "".join(motif),res

def reverseComplementary(seq):
    """Renvoie le brin complémentaire d’une séquence."""
    complementary = {'A': 'T', 'C': 'G', 'G': 'C', 'T':'A'}
    return [complementary[n] for n in seq[::-1]]

In [4]:
Sequences= generateRandomSequences()
print("generated Sequences (seq0): ",Sequences[0])

Pattern=generateRandomPattern()
print("generated Pattern: ",Pattern)

Sequences=implantMotifSubstituions(Sequences,Pattern)
print("Sequences with Pattern ( with substitutions ): ",Sequences[0])

generated Sequences (seq0):  AGGATGGCGGCCGTGCGAGGGTCAATGGAATCTTGTGCACGATGGCTCTTTCCTCCGTTACTACTCTAGAGTGTTCCCGTGCCCCCTGTTTTATTCTAGC
generated Pattern:  TCATTGAT
Sequences with Pattern ( with substitutions ):  AGGATGGTCATTGATCGGCCGTGCGAGGGTCAATGGAATCTTGTGCACGATGGCTCTTTCCTCCGTTACTACTCTAGAGTGTTCCCGTGCCCCCTGTTTTATTCTAGC


In [5]:
def badCharacter(pattern):
    '''
    The preprocessing function for Boyer Moore's bad character heuristic
    '''
    res={"A":1,"T":1,"G":1,"C":1}
    taille=len(pattern)
    for k in range(0,taille):
        res[pattern[k]]=max(1,taille-k-1)
  
    return res


In [6]:
def verif(sequence,pattern):
    taille=len(pattern)
    for i in range(taille-1,-1,-1):
        if sequence[i]!=pattern[i]:
            return sequence[i]
    return 0


def search(sequence, pattern):
    index=0
    taille=len(pattern)
    while index<len_sequences-taille:
        tmp_seq=sequence[index:index+taille]
        
        decalage=verif(tmp_seq,pattern)
        
        if (decalage==0):
            return index
        else:
            index+=badCharacter(tmp_seq)[decalage]
 
    return -1
    

In [7]:
seq_test_A=[ "B" for k in range(0,len_sequences)]
assert(search(seq_test_A, "AAAAAAAA")==-1)

index_debut_test=0
for k in range(0,pattern_len):
    seq_test_A[index_debut_test+k]="A"
    
assert(search(seq_test_A, "AAAAAAAA")==index_debut_test)
assert(search("AABABABCC", "ABABABCC")==1)

In [8]:
#Le nombre de comparaisons inutiles varie en fonction de la séquence générée.
#Cependant la force de l'algo de boyer-moore est le fait qu'on fasse des sauts
#entre chaque comparaisons. 
#théoriquement pour une chaîne de longueur k on ne fera pas k-pattern_len comparaisons comme
#un algorithmme de recherche "naïf" le ferait.

In [9]:
def verif_suffix(pattern,index_sc,index_lp):
    index_lp_tmp=len(pattern)-1
    while (index_sc>=0) and (index_lp_tmp>=index_lp):
        if pattern[index_sc]!=pattern[index_lp_tmp]:
            return False
        index_sc+=-1
        index_lp_tmp+=-1
    return True

In [10]:
assert(verif_suffix("ABAAAABA",0,6)==True)
assert(verif_suffix("ABAAAABA",2,6)==True)
assert(verif_suffix("CBAAAABA",0,6)==False)

In [11]:
def find_suffix(pattern,index_lp):
    for k in range(len(pattern)-2,-1,-1):
        if verif_suffix(pattern,k,index_lp):
            return k+1
    return 0

In [12]:
assert(find_suffix("ABAAAABA",6)==3) #BA
assert(find_suffix("CBAAAABA",5)==0) #ABA

In [13]:
def goodSuffix(pattern):
    
    res = dict()
    taille=len(pattern)
    for k in range(taille-1,0,-1):
        res[pattern[k:]]=taille-find_suffix(pattern,k)
    
    return res

In [14]:
assert ( goodSuffix("BAOBAB")=={'B': 2, 'AB': 5, 'BAB': 5, 'OBAB': 5, 'AOBAB': 5} )
assert (goodSuffix("ABCBAB") =={'B': 2, 'AB': 4, 'BAB': 4, 'CBAB': 4, 'BCBAB': 4} )

In [15]:
def search(sequence, pattern):
    '''
    A pattern searching function that uses Bad Character
    Heuristic of Boyer Moore Algorithm
    '''
    index=pattern_len-1
    taille=len(sequence)
    
    badChar = badCharacter(pattern) 
    gs = goodSuffix(pattern)
    
    while index<taille:
        
        tmp_pattern=sequence[index-pattern_len+1:index+1]
        for i in range(pattern_len-1,-1,-1):
            
            if tmp_pattern[i]!=pattern[i]:
                
                index+=max(gs[pattern[i]],badChar[pattern[i]])
                break
            if (i==0):
                return index-pattern_len+1
    return -1
        
    badChar = badCharacter(motif) 
    gs = goodSuffix(motif)

In [16]:
assert(search("ACAAABABABCC", "ABABABCC")==4)
assert(search("BBBBBBBBBBBBBB", "ABABABCC")==-1)
assert(search("ABABABCC", "ABABABCC")==0)
assert(search("AABABABCC", "ABABABCC")==1)

In [18]:
def indexTable(k, texte):
    indexes  = dict();
    
    taille_texte=len(texte)
    for i in range(0,taille_texte-k+1):
        mot_tmp=''.join(texte[i:i+k])
        if mot_tmp in indexes:
            indexes[mot_tmp].append(i)
        else:
            indexes[mot_tmp]=[i]
        
    return indexes


In [19]:
def seeds(k, motif):
    res=set()
    taille_motif=len(motif)
    
    for i in range(0,taille_motif-k+1):
        res.add(motif[i:i+k])
    return res


seeds(3,"ATGATG")

{'ATG', 'GAT', 'TGA'}

In [20]:
def searchIndexTable(indexTable, k, motif):
    kmer = seeds(k, motif)
    indexes = []
    if kmer != {}:
        for k in kmer:
            if k in indexTable:
                indexes += indexTable[k]
    return indexes

In [21]:
indexT = indexTable(8, ''.join(sequences))
print(searchIndexTable(indexT, 8, 'TCCGTGGA'))

[1293, 3171, 6622, 6750, 7204, 7566, 8396, 9444, 10914, 11752, 18924, 19659, 20229, 20245, 23155, 25941, 26893, 28604, 29482, 31471, 35209, 35849, 37229, 37391, 38058]
